In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [8]:
df=pd.read_excel('./data/Online_Retail.xlsx')

In [9]:
df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

In [10]:
df1=df

In [11]:
df1.Country.nunique()

38

In [12]:
df1.Country.unique()

array([u'United Kingdom', u'France', u'Australia', u'Netherlands',
       u'Germany', u'Norway', u'EIRE', u'Switzerland', u'Spain', u'Poland',
       u'Portugal', u'Italy', u'Belgium', u'Lithuania', u'Japan',
       u'Iceland', u'Channel Islands', u'Denmark', u'Cyprus', u'Sweden',
       u'Austria', u'Israel', u'Finland', u'Bahrain', u'Greece',
       u'Hong Kong', u'Singapore', u'Lebanon', u'United Arab Emirates',
       u'Saudi Arabia', u'Czech Republic', u'Canada', u'Unspecified',
       u'Brazil', u'USA', u'European Community', u'Malta', u'RSA'], dtype=object)

In [13]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()

customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)


Country  CustomerID
36        United Kingdom        3950
14               Germany          95
13                France          87
31                 Spain          31
3                Belgium          25
33           Switzerland          21
27              Portugal          19
19                 Italy          15
12               Finland          12
1                Austria          11
25                Norway          10
24           Netherlands           9
0              Australia           9
6        Channel Islands           9
9                Denmark           9
7                 Cyprus           8
32                Sweden           8
20                 Japan           8
26                Poland           6
34                   USA           4
5                 Canada           4
37           Unspecified           4
18                Israel           4
15                Greece           4
10                  EIRE           3
23                 Malta           2
35  United Arab Emirates           2
2                Bahrain           2
22             Lithuania           1
8         Czech Republic           1
21               Lebanon           1
28                   RSA           1
29          Saudi Arabia           1
30             Singapore           1
17               Iceland           1
4                 Brazil           1
11    European Community           1
16             Hong Kong           0

In [14]:
df1=df1.loc[df1['Country'] == 'United Kingdom']

In [15]:
df1.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [18]:
df1=df1[pd.notnull(df1['CustomerID'])]

In [19]:
df1=df1[pd.notnull(df1['CustomerID'])]

In [20]:
df1.Quantity.min()

-80995

In [21]:
df1 = df1[(df1['Quantity'] > 0)]

In [23]:
df1.shape

(354345, 8)

In [24]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [25]:
def unique_counts(df1):
    for i in df1.columns:
        count=df1[i].nunique()
        print(i, ":", count)
        
unique_counts(df1)

(u'InvoiceNo', ':', 16649)
(u'StockCode', ':', 3645)
(u'Description', ':', 3844)
(u'Quantity', ':', 294)
(u'InvoiceDate', ':', 15615)
(u'UnitPrice', ':', 403)
(u'CustomerID', ':', 3921)
(u'Country', ':', 1)


In [26]:
df1['TotalPrice']=df1['Quantity']*df1['UnitPrice']

In [27]:
df1['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [28]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [32]:
import datetime as dt
NOW=dt.datetime(2011,12,10)

df1['InvoiceDate'] =pd.to_datetime(df1['InvoiceDate'])

In [34]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

In [35]:
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [36]:
rfmTable.head()

frequency  monetary_value  recency
CustomerID                                    
12346.0             1        77183.60      325
12747.0           103         4196.01        2
12748.0          4596        33719.73        0
12749.0           199         4090.88        3
12820.0            59          942.34        3

In [37]:
first_customer=df1[df1['CustomerID'] == 12346.0]
first_customer

InvoiceNo StockCode                     Description  Quantity  \
61619    541431     23166  MEDIUM CERAMIC TOP STORAGE JAR     74215   

              InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
61619 2011-01-18 10:01:00       1.04     12346.0  United Kingdom     77183.6

In [38]:
quantiles=rfmTable.quantile(q=[0.25,0.50,0.75])
quantiles=quantiles.to_dict()

In [39]:
segmented_rfm=rfmTable

In [40]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    
    elif x <= d[p][0.50]:
        return 2
    
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4
    

In [41]:
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
        
        
        
        

In [43]:
segmented_rfm['r_quartile']=segmented_rfm['recency'].apply(RScore,args=('recency',quantiles,))

segmented_rfm['f_quartile']=segmented_rfm['frequency'].apply(FMScore,args=('frequency',quantiles,))

segmented_rfm['m_quartile']=segmented_rfm['monetary_value'].apply(FMScore,args=('monetary_value',quantiles,))

segmented_rfm.head()

frequency  monetary_value  recency  r_quartile  f_quartile  \
CustomerID                                                               
12346.0             1        77183.60      325           4           4   
12747.0           103         4196.01        2           1           1   
12748.0          4596        33719.73        0           1           1   
12749.0           199         4090.88        3           1           1   
12820.0            59          942.34        3           1           2   

            m_quartile  
CustomerID              
12346.0              1  
12747.0              1  
12748.0              1  
12749.0              1  
12820.0              2

In [44]:
segmented_rfm['RFMScore']=segmented_rfm.r_quartile.map(str)+segmented_rfm.f_quartile.map(str)+segmented_rfm.m_quartile.map(str)

segmented_rfm.head()

frequency  monetary_value  recency  r_quartile  f_quartile  \
CustomerID                                                               
12346.0             1        77183.60      325           4           4   
12747.0           103         4196.01        2           1           1   
12748.0          4596        33719.73        0           1           1   
12749.0           199         4090.88        3           1           1   
12820.0            59          942.34        3           1           2   

            m_quartile RFMScore  
CustomerID                       
12346.0              1      441  
12747.0              1      111  
12748.0              1      111  
12749.0              1      111  
12820.0              2      122

In [45]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value',ascending=False).head(10)

frequency  monetary_value  recency  r_quartile  f_quartile  \
CustomerID                                                               
18102.0           431       259657.30        0           1           1   
17450.0           337       194550.79        8           1           1   
17511.0           963        91062.38        2           1           1   
16684.0           277        66653.56        4           1           1   
14096.0          5111        65164.79        4           1           1   
13694.0           568        65039.62        3           1           1   
15311.0          2379        60767.90        0           1           1   
13089.0          1818        58825.83        2           1           1   
15769.0           130        56252.72        7           1           1   
15061.0           403        54534.14        3           1           1   

            m_quartile RFMScore  
CustomerID                       
18102.0              1      111  
17450.0              1      111  
17511.0              1      111  
16684.0              1      111  
14096.0              1      111  
13694.0              1      111  
15311.0              1      111  
13089.0              1      111  
15769.0              1      111  
15061.0              1      111